# Quick-start

This document helps you get up-and-running with `alr` immediately.
It should give you a general idea of how to get started with
this package.


In [1]:
import numpy as np
import torch
import torch.utils.data as torchdata
import tqdm
import sys

from torch.nn import functional as F
from torch import nn

from alr import MCDropout
from alr.acquisition import RandomAcquisition
from alr.utils import stratified_partition
from alr.data import DataManager, UnlabelledDataset
from alr.data.datasets import Dataset

np.random.seed(42)
torch.manual_seed(42)
data_loader_params = dict(pin_memory=True, num_workers=2, batch_size=32)
device = torch.device('gpu:0' if torch.cuda.is_available() else 'cpu')

Firstly, we load and prepare our data.
Note that we partitioned the training set into labelled and unlabelled sets
using `stratified partition` which balances the number of classes in the training pool:

In [2]:
# load training data
train, test = Dataset.MNIST.get()
pool, train = stratified_partition(train, Dataset.MNIST.about.n_class, 20)
pool = UnlabelledDataset(pool)
len(train), len(test), len(pool)

(20, 10000, 59980)

`MCDropout` lets us define a Bayesian NN. It provides an implementation
for `stochastic_forward` which we will use in the next section for the
acquisition function.

> Notice the dropout layers have been changed to their `Persistent` versions.

In [3]:
# instantiate a regular model and an optimiser.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 1024)
        self.drop1 = nn.Dropout()
        self.fc2 = nn.Linear(1024, 2048)
        self.drop2 = nn.Dropout()
        self.fc3 = nn.Linear(2048, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.drop1(self.fc1(x)))
        x = F.relu(self.drop2(self.fc2(x)))
        return self.fc3(x)

model = MCDropout(Net(), forward=10).to(device)
optimiser = torch.optim.Adam(model.parameters())
model

MCDropout(
  (base_model): Net(
    (fc1): Linear(in_features=784, out_features=1024, bias=True)
    (drop1): PersistentDropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=1024, out_features=2048, bias=True)
    (drop2): PersistentDropout(p=0.5, inplace=False)
    (fc3): Linear(in_features=2048, out_features=10, bias=True)
  )
)

Now, we can instantiate an acquisition function
and an associated `DataManager` instance:

In [4]:
ra = RandomAcquisition()
dm = DataManager(train, pool, ra)

Here, we define the usual training and evaluation loops:

In [5]:
def train(model: nn.Module,
          dataloader: torchdata.DataLoader,
          optimiser: torch.optim.Optimizer,
          epochs: int = 50):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    tepochs = tqdm.trange(epochs, file=sys.stdout)
    for _ in tepochs:
        epoch_losses = []
        for x, y in dataloader:
            if device:
                x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            epoch_losses.append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        losses.append(np.mean(epoch_losses))
        tepochs.set_postfix(loss=losses[-1])
    return losses


def evaluate(model: MCDropout,
             dataloader: torchdata.DataLoader) -> float:
    model.eval()
    score = total = 0
    with torch.no_grad():
        for x, y in dataloader:
            if device:
                x, y = x.to(device), y.to(device)
            _, preds = torch.max(model.predict(x), dim=1)
            score += (preds == y).sum().item()
            total += y.size(0)

    return score / total

Finally, we can put everything together:

In [6]:
ITERS = 10
EPOCHS = 10
accs = {}

# In each iteration, acquire 10 points at once and re-evaluate the model
for i in range(ITERS):
    print(f"Acquisition iteration {i + 1} ({(i + 1) / ITERS:.2%}), "
          f"training size: {dm.n_labelled}")
    model.reset_weights()
    train(model, dataloader=torchdata.DataLoader(dm.labelled, **data_loader_params),
          optimiser=optimiser, epochs=EPOCHS)
    accs[dm.n_labelled] = evaluate(model, torchdata.DataLoader(test,
                                                               **data_loader_params))
    print(f"Accuracy = {accs[dm.n_labelled]}\n=====")
    dm.acquire(b=10)
accs

Acquisition iteration 1 (10.00%), training size: 20
100%|██████████| 10/10 [00:01<00:00,  6.93it/s, loss=0.0134]
Accuracy = 0.5313
=====
Acquisition iteration 2 (20.00%), training size: 30
100%|██████████| 10/10 [00:01<00:00,  7.17it/s, loss=0.027] 
Accuracy = 0.5548
=====
Acquisition iteration 3 (30.00%), training size: 40
100%|██████████| 10/10 [00:01<00:00,  6.24it/s, loss=0.0332]
Accuracy = 0.5606
=====
Acquisition iteration 4 (40.00%), training size: 50
100%|██████████| 10/10 [00:01<00:00,  6.09it/s, loss=0.0664]
Accuracy = 0.6117
=====
Acquisition iteration 5 (50.00%), training size: 60
100%|██████████| 10/10 [00:01<00:00,  6.16it/s, loss=0.00393]
Accuracy = 0.6412
=====
Acquisition iteration 6 (60.00%), training size: 70
100%|██████████| 10/10 [00:01<00:00,  5.42it/s, loss=0.0807]
Accuracy = 0.6809
=====
Acquisition iteration 7 (70.00%), training size: 80
100%|██████████| 10/10 [00:01<00:00,  5.31it/s, loss=0.0062]
Accuracy = 0.6931
=====
Acquisition iteration 8 (80.00%), traini

{20: 0.5313,
 30: 0.5548,
 40: 0.5606,
 50: 0.6117,
 60: 0.6412,
 70: 0.6809,
 80: 0.6931,
 90: 0.6961,
 100: 0.7122,
 110: 0.7178}